# Laboratorio de regresión - 5 -pt2

|                |   |
:----------------|---|
| **Nombre**     |  Jesús Alejandro García García |
| **Fecha**      |  15-09-25 |
| **Expediente** |  751749 |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Porque necesitamos asegurar que no se realize un overfit

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Si, pero entonces tenemos que encontrar una manera de evitar la generalizacion

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [14]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df=pd.read_excel('Motor Trend Car Road Tests.xlsx')
df.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [3]:
df.drop(columns=['model'], inplace=True)
df.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [4]:
df.describe()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.0000
mean,20.090625,6.187500,230.721875,146.687500,3.596563,3.217250,17.848750,0.437500,0.406250,3.687500,2.8125
std,6.026948,1.785922,123.938694,68.562868,0.534679,0.978457,1.786943,0.504016,0.498991,0.737804,1.6152
min,10.400000,4.000000,71.100000,52.000000,2.760000,1.513000,14.500000,0.000000,0.000000,3.000000,1.0000
25%,15.425000,4.000000,120.825000,96.500000,3.080000,2.581250,16.892500,0.000000,0.000000,3.000000,2.0000
50%,19.200000,6.000000,196.300000,123.000000,3.695000,3.325000,17.710000,0.000000,0.000000,4.000000,2.0000
75%,22.800000,8.000000,326.000000,180.000000,3.920000,3.610000,18.900000,1.000000,1.000000,4.000000,4.0000
max,33.900000,8.000000,472.000000,335.000000,4.930000,5.424000,22.900000,1.000000,1.000000,5.000000,8.0000


In [5]:
df.drop_duplicates(inplace=True)
df.isnull().sum()/len(df)

mpg     0.0
cyl     0.0
disp    0.0
hp      0.0
drat    0.0
wt      0.0
qsec    0.0
vs      0.0
am      0.0
gear    0.0
carb    0.0
dtype: float64

Mi intento antes de la mejora del profe 
for i in range(32):

    test = [i]  

    train = []

    for j in range(32):

        if j != i:

            train.append(j)

    train_df = df.iloc[train]
    
    test_df = df.iloc[test]

In [6]:
mses=[]
for i in range(32):
    
    train_df = df.drop(i)
    test_df = df.iloc[[i]]


    cols=df.columns
    cols=cols.drop("mpg")
    x_train_list = []
    x_test_list = []

    for col in cols:
        x_train_list.append(train_df[col].values.reshape(-1, 1))
        x_test_list.append(test_df[col].values.reshape(-1, 1))

    X_train = np.hstack(x_train_list)
    X_test = np.hstack(x_test_list)
    y_train = train_df["mpg"].values.reshape(-1, 1)
    y_test = test_df["mpg"].values.reshape(-1, 1)

    scaler=StandardScaler().fit(X_train)
    Xscaled_train=scaler.transform(X_train)

    x=np.ones((len(X_train),1))
    X_e_train=np.hstack((x,Xscaled_train))

    ols=sm.OLS(y_train,X_e_train)
    results=ols.fit()

    Xscaled_test = scaler.transform(X_test)
    x=np.ones((len(X_test),1))
    X_e_test=np.hstack((x,Xscaled_test))

    y_pred=results.predict(X_e_test)
    mse=mean_squared_error(y_test,y_pred)
    mses.append(mse)

print("MSE promedio:", np.mean(mses))
print("Desviación estándar del MSE:", np.std(mses))

MSE promedio: 12.181558006901934
Desviación estándar del MSE: 17.067399871888544


Interpreta.

Como tiene una desviacion estandar alta en su MSE podemos asumir que varia mucho el error por modelo. 

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [39]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [43]:

df = pd.DataFrame(housing.data, columns=housing.feature_names)
df["y"] = housing.target
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,y
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [ ]:
df = df.sample(frac=1, random_state=47)

In [50]:
n=len(df)
X = df.drop(columns=['y']).values.reshape(-1,1)
y = df['y'].values
fold=n//10

mse_list=[]
for i in range(10):
    
    start = i * fold
    end = (i + 1) * fold
    test_df = np.arange(start, end)
    train_df=np.setdiff1d(np.arange(n), test_df)

    train_df = df.iloc[train_df]
    test_df = df.iloc[test_df]

    cols=df.columns
    cols=cols.drop("y")
    X_train = train_df[cols].values
    X_test = test_df[cols].values
    y_train = train_df["y"].values.reshape(-1, 1)
    y_test = test_df["y"].values.reshape(-1, 1)

    scaler=StandardScaler().fit(X_train)
    Xscaled_train=scaler.transform(X_train)

    x=np.ones((len(X_train),1))
    X_e_train=np.hstack((x,Xscaled_train))

    ols=sm.OLS(y_train,X_e_train)
    results=ols.fit()

    Xscaled_test = scaler.transform(X_test)
    x=np.ones((len(X_test),1))
    X_e_test=np.hstack((x,Xscaled_test))

    y_pred=results.predict(X_e_test)
    mse=mean_squared_error(y_test,y_pred)
    mse_list.append(mse)

print("MSE promedio:", np.mean(mse_list))
print("Desviación estándar del MSE:", np.std(mse_list))
    

MSE promedio: 0.5286891563660314
Desviación estándar del MSE: 0.0321757690169523


Interpreta.

Podemos generalizar y crear un modelo con todos los datos

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3